# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib .pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
ruta = "/content/dataset_banco_clean.csv"

data = pd.read_csv(ruta)

data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


In [ ]:
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45189 entries, 0 to 45188
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        45189 non-null  int64  
 1   job        45189 non-null  object 
 2   marital    45189 non-null  object 
 3   education  45189 non-null  object 
 4   default    45189 non-null  object 
 5   balance    45189 non-null  float64
 6   housing    45189 non-null  object 
 7   loan       45189 non-null  object 
 8   contact    45189 non-null  object 
 9   day        45189 non-null  int64  
 10  month      45189 non-null  object 
 11  duration   45189 non-null  float64
 12  campaign   45189 non-null  int64  
 13  pdays      45189 non-null  float64
 14  previous   45189 non-null  int64  
 15  poutcome   45189 non-null  object 
 16  y          45189 non-null  object 
dtypes: float64(3), int64(4), object(10)
memory usage: 5.9+ MB
None


# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

## Normalización

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.compose import ColumnTransformer

# Definir las columnas numéricas y categóricas
numeric = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical = ['marital', 'housing', 'loan', 'contact', 'job', 'month', 'poutcome', 'education']

# Definir transformaciones para variables numéricas y categóricas
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Utilizar ColumnTransformer para aplicar transformaciones a las columnas correspondientes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric),
        ('cat', categorical_transformer, categorical)
    ])

# Aplicar el preprocesador a las características de entrenamiento, validación y prueba
x_train_s = preprocessor.fit_transform(x_train)
x_val_s = preprocessor.transform(x_val)
x_test_s = preprocessor.transform(x_test)

# Binarizar la variable objetivo
label_binarizer = LabelBinarizer()
y_train_bin = label_binarizer.fit_transform(y_train)
y_val_bin = label_binarizer.transform(y_val)
y_test_bin = label_binarizer.transform(y_test)

print("Valores mínimos y máximos de x_train_s:")
print(x_train_s.min(axis=0), x_train_s.max(axis=0))
print("\nValores mínimos y máximos de x_val_s:")
print(x_val_s.min(axis=0), x_val_s.max(axis=0))
print("\nValores mínimos y máximos de x_test_s:")
print(x_test_s.min(axis=0), x_test_s.max(axis=0))


Valores mínimos y máximos de x_train_s:
[-2.17488913 -2.2272257  -1.77949291 -1.01046087 -0.57420934 -0.41226409
 -0.30457273  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.        ] [  5.11950601 124.71889585   1.8261209   14.17629239  17.8661089
   8.30661788  26.60083011   1.           1.           1.
   1.           1.           1.           1.           1.
   1.           1.           1.           1.           1.
   1.           1.           1.           1.           1.
   1.           1.           1.           1.           1.
   1.           1.           1.           1.           1.
   1.

## Particiones

In [ ]:
x = data.drop(['y'], axis = 1)
y = data['y']

'''Crear los sets de entrenamiento (70%), validación (15%)
    y prueba (15%)'''

x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, test_size=0.5, random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_val.shape, y_val.shape)

(31632, 16) (31632,)
(6778, 16) (6778,)
(6779, 16) (6779,)


## Modelos

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
random_forest.fit(x_train_s, y_train)

# Hacer predicciones sobre los datos de validación
y_pred_val = random_forest.predict(x_val_s)

# Evaluar el rendimiento del modelo en los datos de validación
accuracy_val = random_forest.score(x_val_s, y_val)
print("Precisión en los datos de validación:", accuracy_val)

print("Classification Report:")
print(classification_report(y_val, y_pred_val))


Precisión en los datos de validación: 0.9055907951025225
Classification Report:
              precision    recall  f1-score   support

          no       0.92      0.97      0.95      5981
         yes       0.66      0.40      0.50       798

    accuracy                           0.91      6779
   macro avg       0.79      0.69      0.73      6779
weighted avg       0.89      0.91      0.90      6779



### Decison Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(random_state=42)

# Entrenar el modelo con los datos de entrenamiento
decision_tree.fit(x_train_s, y_train)

# Hacer predicciones sobre los datos de validación
y_pred_val = decision_tree.predict(x_val_s)

# Evaluar el rendimiento del modelo en los datos de validación
accuracy_val = decision_tree.score(x_val_s, y_val)
print("Precisión en los datos de validación:", accuracy_val)

print("Classification Report:")
print(classification_report(y_val, y_pred_val))


Precisión en los datos de validación: 0.8808083788169346
Classification Report:
              precision    recall  f1-score   support

          no       0.93      0.93      0.93      5981
         yes       0.49      0.50      0.50       798

    accuracy                           0.88      6779
   macro avg       0.71      0.71      0.71      6779
weighted avg       0.88      0.88      0.88      6779



### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=3)

# Entrenar el modelo con los datos de entrenamiento
knn_classifier.fit(x_train_s, y_train)

# Hacer predicciones sobre los datos de validación
y_pred_val = knn_classifier.predict(x_val_s)

# Evaluar el rendimiento del modelo en los datos de validación
accuracy_val = knn_classifier.score(x_val_s, y_val)
print("Precisión en los datos de validación:", accuracy_val)

print("Classification Report:")
print(classification_report(y_val, y_pred_val))


Precisión en los datos de validación: 0.8883316123322024
Classification Report:
              precision    recall  f1-score   support

          no       0.92      0.96      0.94      5981
         yes       0.54      0.35      0.43       798

    accuracy                           0.89      6779
   macro avg       0.73      0.66      0.68      6779
weighted avg       0.87      0.89      0.88      6779



### SVM

In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel='linear', random_state=42)

# Entrenar el modelo con los datos de entrenamiento
svm_classifier.fit(x_train_s, y_train)

# Hacer predicciones sobre los datos de validación
y_pred_val = svm_classifier.predict(x_val_s)

# Evaluar el rendimiento del modelo en los datos de validación
accuracy_val = svm_classifier.score(x_val_s, y_val)
print("Precisión en los datos de validación:", accuracy_val)

print("Classification Report:")
print(classification_report(y_val, y_pred_val))

Precisión en los datos de validación: 0.8960023602301225
Classification Report:
              precision    recall  f1-score   support

          no       0.90      0.99      0.94      5981
         yes       0.70      0.20      0.31       798

    accuracy                           0.90      6779
   macro avg       0.80      0.60      0.63      6779
weighted avg       0.88      0.90      0.87      6779



### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradient_boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
gradient_boosting.fit(x_train_s, y_train)

# Hacer predicciones sobre los datos de validación
y_pred_val = gradient_boosting.predict(x_val_s)

# Evaluar el rendimiento del modelo en los datos de validación
accuracy_val = gradient_boosting.score(x_val_s, y_val)
print("Precisión en los datos de validación:", accuracy_val)

print("Classification Report:")
print(classification_report(y_val, y_pred_val))

Precisión en los datos de validación: 0.9064758813984364
Classification Report:
              precision    recall  f1-score   support

          no       0.93      0.97      0.95      5981
         yes       0.66      0.42      0.51       798

    accuracy                           0.91      6779
   macro avg       0.79      0.69      0.73      6779
weighted avg       0.90      0.91      0.90      6779



## Comparación modelos

Basándonos en la comparativa de los modelos de Random Forest, Decision Tree, k-NN, SVM y Gradient Boosting en función de la precisión en los datos de validación y el informe de clasificación, podemos extraer las siguientes conclusiones:

1. **Precisión en los datos de validación**: Los modelos de Random Forest y Gradient Boosting muestran la mayor precisión en los datos de validación, con valores de alrededor del 90.6%. Estos modelos superan ligeramente a los modelos SVM y k-NN, que tienen una precisión en el rango de 88.8% a 89.6%. El modelo Decision Tree tiene la precisión más baja, alrededor del 88.1%.

2. **Precision y Recall**: Tanto Random Forest como Gradient Boosting muestran un buen equilibrio entre precisión y recall para ambas clases (sí y no). Tienen una precision y recall aceptables para ambas clases. Por otro lado, SVM muestra una buena precisión para la clase negativa, pero su recall para la clase positiva es bastante bajo, lo que indica que podría estar teniendo dificultades para identificar los casos positivos. El modelo Decision Tree también muestra un desequilibrio entre precision y recall para la clase positiva.

3. **F1-score**: El F1-score, que es una medida del balance entre precisión y recall, es alto para Random Forest y Gradient Boosting en comparación con los otros modelos. Esto sugiere que estos modelos logran un buen equilibrio entre la precisión y el recall para ambas clases.

En conclusión, tanto Random Forest como Gradient Boosting parecen ser las mejores opciones para este problema de clasificación en particular, ya que muestran una mayor precisión y un buen equilibrio entre precisión y recall para ambas clases. Sin embargo, la elección final del modelo dependerá de otros factores como la interpretabilidad del modelo, el tiempo de entrenamiento y la escalabilidad.

## Ensembles

In [30]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report


# Definir los clasificadores individuales
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
gradient_boosting = GradientBoostingClassifier(n_estimators=100, random_state=42)
svm_classifier = SVC(kernel='linear', probability=True, random_state=42)  # Se establece probability=True para que SVM pueda estimar probabilidades

# Crear el ensemble mediante el método de votación
ensemble = VotingClassifier(estimators=[('rf', random_forest), ('gb', gradient_boosting), ('svm', svm_classifier)], voting='soft')

# Entrenar el ensemble con los datos de entrenamiento
ensemble.fit(x_train_s, y_train)

# Hacer predicciones sobre los datos de validación
y_pred_val = ensemble.predict(x_val_s)

# Evaluar el rendimiento del ensemble en los datos de validación
accuracy_val = ensemble.score(x_val_s, y_val)
print("Precisión en los datos de validación:", accuracy_val)
print("Classification Report:")
print(classification_report(y_val, y_pred_val))



Precisión en los datos de validación: 0.9002802773270394
              precision    recall  f1-score   support

          no       0.91      0.99      0.95      5981
         yes       0.75      0.23      0.35       798

    accuracy                           0.90      6779
   macro avg       0.83      0.61      0.65      6779
weighted avg       0.89      0.90      0.88      6779



La precisión en los datos de validación resultó ser del 90.03%, lo que indica que el ensemble tiene un buen rendimiento en general. Sin embargo, al observar el informe de clasificación, se puede apreciar que la precisión para la clase 'yes' es relativamente baja (75%), y el valor de f1-score también es bajo para esta clase (0.35). Esto sugiere que el modelo tiene dificultades para predecir correctamente la clase 'yes'.

## Conclusiones

En este notebook no ha hecho falta hacer una limpieza del dataset puesto que partiamos de uno previamente limpiado, en esta limpieza se llevarían a cabo tareas como ver si hay datos faltantes en algunas celdas, columnas irrelevantes, registros repetidos, errores tipográficos, etc. Otra tarea a realizar en el preprocesado es la normalización de características, consiste en transformar las características del dataset a una escala común, de modo que todas las características tengan una distribución similar. Esto tiene diversos beneficios que impactan positivamente en el rendimiento de los modelos de aprendizaje automático.

Hemos utilizado diferentes algoritmos de clasificación para poder observar cual es el que nos proporcionaba un mayor rendimiento, como por ejemplo, Random Forest, kNN, SVM, Gradient Boosting, han sido algunos de los utlizados.

Evaluar adecuadamente el desempeño de los modelos de clasificación es esencial para entender su efectividad en la tarea que realizan. Una forma común de hacerlo es mediante el uso de métricas que nos brindan información sobre diferentes aspectos de su rendimiento.Por ejemplo, la precisión nos indica qué tan exacto es el modelo al predecir las clases correctamente. El F1-score, por otro lado, considera tanto la precisión como la exhaustividad del modelo, proporcionando una medida más equilibrada cuando las clases están desbalanceadas.